# Debugger and Profiler

In [2]:
# install dependencies
!pip install smdebug

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached smdebug-1.0.12-py2.py3-none-any.whl (270 kB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp37-cp37m-manylinux2010_x86_64.whl (20 kB)


In [3]:
hyperparameters = {
    "batch_size": 2048,
    "gpu": True,
    "epoch": 2,
    "model": "resnet50",
}

In [4]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
] #TODO: Can you add the rules you want to track

In [5]:
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile

#TODO: Can you create the profiler and debugger configs
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)
debugger_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)

In [10]:
import sagemaker
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    source_dir="scripts",
    entry_point="pytorch_cifar_profiling.py",
    framework_version="1.8",
    py_version="py36",
    hyperparameters=hyperparameters,
    profiler_config=profiler_config,
    rules=rules,
    debugger_hook_config=debugger_config,
)

In [11]:
estimator.fit(wait=True)

2021-12-19 03:36:18 Starting - Starting the training job...
2021-12-19 03:36:43 Starting - Launching requested ML instancesVanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LossNotDecreasing: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
......
2021-12-19 03:37:43 Starting - Preparing the instances for training.........
2021-12-19 03:39:16 Downloading - Downloading input data
2021-12-19 03:39:16 Training - Downloading the training image.........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-12-19 03:43:19,142 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-12-19 03:43:19,166 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-12-19 03:43:20,589 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-12-19

In [12]:
import boto3

session = boto3.session.Session()
region = session.region_name

training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

Training jobname: pytorch-training-2021-12-19-03-36-17-797
Region: us-east-1


In [13]:
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())

[2021-12-19 03:59:24.099 datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395:36 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-12-19 03:59:24.114 datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395:36 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-1-755391689112/pytorch-training-2021-12-19-03-36-17-797/debug-output


In [14]:
# TODO: Can you print the names of all the tensors that were tracked
print(trial.tensor_names())

[2021-12-19 03:59:26.272 datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395:36 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2021-12-19 03:59:27.295 datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395:36 INFO trial.py:210] Loaded all steps
['CrossEntropyLoss_output_0', 'gradient/ResNet_bn1.bias', 'gradient/ResNet_bn1.weight', 'gradient/ResNet_conv1.weight', 'gradient/ResNet_fc.bias', 'gradient/ResNet_fc.weight', 'gradient/ResNet_layer1.0.bn1.bias', 'gradient/ResNet_layer1.0.bn1.weight', 'gradient/ResNet_layer1.0.bn2.bias', 'gradient/ResNet_layer1.0.bn2.weight', 'gradient/ResNet_layer1.0.bn3.bias', 'gradient/ResNet_layer1.0.bn3.weight', 'gradient/ResNet_layer1.0.conv1.weight', 'gradient/ResNet_layer1.0.conv2.weight', 'gradient/ResNet_layer1.0.conv3.weight', 'gradient/ResNet_layer1.0.downsample.0.weight', 'gradient/ResNet_layer1.0.downsample.1.bias', 'gradient/ResNet_layer1.0.downsample.1.weight', 'gradient/ResNet_layer1.1.bn1.bias', 'gradient

In [18]:
# TODO: Can you print the number of datapoints for one of those tensors
# for both train and eval mode
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN)))
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL)))

1
1


In [19]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-east-1-755391689112/', 'ProfilingIntervalInMilliseconds': 500, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'FileOpenFailThreshold': '50', 'HorovodProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'LocalPath': '/opt/ml/output/profiler', 'PythonProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760', 'SMDataParallelProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }'}}
s3 path:s3://sagemaker-us-east-1-755391689112/pytorch-training-2021-12-19-03-36-17-797/profiler-output


Profiler data from system is available


In [20]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

[2021-12-19 04:02:48.052 datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395:36 INFO metrics_reader_base.py:134] Getting 19 event files
select events:['total']
select dimensions:['CPU', 'GPU']
filtered_events:{'total'}
filtered_dimensions:{'GPUUtilization-nodeid:algo-1', 'GPUMemoryUtilization-nodeid:algo-1', 'CPUUtilization-nodeid:algo-1'}


In [21]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

You will find the profiler report in s3://sagemaker-us-east-1-755391689112/pytorch-training-2021-12-19-03-36-17-797/rule-output


In [22]:
! aws s3 ls {rule_output_path} --recursive

2021-12-19 03:58:19     413436 pytorch-training-2021-12-19-03-36-17-797/rule-output/ProfilerReport/profiler-output/profiler-report.html
2021-12-19 03:58:18     269048 pytorch-training-2021-12-19-03-36-17-797/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2021-12-19 03:58:13        558 pytorch-training-2021-12-19-03-36-17-797/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2021-12-19 03:58:13      28478 pytorch-training-2021-12-19-03-36-17-797/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2021-12-19 03:58:13       1936 pytorch-training-2021-12-19-03-36-17-797/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2021-12-19 03:58:13        130 pytorch-training-2021-12-19-03-36-17-797/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
2021-12-19 03:58:13      11473 pytorch-training-2021-12-19-03-36-17-797/rule-output/ProfilerReport/profiler-output/profiler-re

In [23]:
! aws s3 cp {rule_output_path} ./ --recursive

download: s3://sagemaker-us-east-1-755391689112/pytorch-training-2021-12-19-03-36-17-797/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json to ProfilerReport/profiler-output/profiler-reports/BatchSize.json
download: s3://sagemaker-us-east-1-755391689112/pytorch-training-2021-12-19-03-36-17-797/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json to ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
download: s3://sagemaker-us-east-1-755391689112/pytorch-training-2021-12-19-03-36-17-797/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json to ProfilerReport/profiler-output/profiler-reports/Dataloader.json
download: s3://sagemaker-us-east-1-755391689112/pytorch-training-2021-12-19-03-36-17-797/rule-output/ProfilerReport/profiler-output/profiler-report.html to ProfilerReport/profiler-output/profiler-report.html
download: s3://sagemaker-us-east-1-755391689112/pytorch-training-2021-12-19-03-36-17

In [24]:
import os

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

In [25]:
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")